# Amazon Bedrock AgentCore Runtime での MCP サーバーのホスティング

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime で MCP（Model Context Protocol）サーバーをホストする方法を学習します。Amazon Bedrock AgentCore Python SDK を使用して、MCP ツールを Amazon Bedrock AgentCore と互換性のある MCP サーバーとしてラップします。

Amazon Bedrock AgentCore Python SDK は MCP サーバーの実装詳細を処理するため、ツールの核となる機能に集中できます。コードを AgentCore の標準化された MCP プロトコル契約に変換し、直接通信を可能にします。

### チュートリアル詳細

| 項目                | 詳細                                                      |
|:-------------------|:----------------------------------------------------------|
| チュートリアル タイプ | ツールのホスティング                                       |
| ツール タイプ        | MCP server                                               |
| チュートリアル コンポーネント | AgentCore Runtime での MCP サーバーのホスティング     |
| チュートリアル領域   | 横断的                                                    |
| サンプルの複雑さ     | 簡単                                                      |
| 使用 SDK           | Amazon BedrockAgentCore Python SDK and MCP               |

### チュートリアル アーキテクチャ

このチュートリアルでは、AgentCore runtime に MCP サーバーをデプロイする方法について説明します。

デモンストレーション目的として、3つのツール（`add_numbers`、`multiply_numbers`、`greet_user`）を持つシンプルな MCP サーバーを使用します。

<div style="text-align:left">
    <img src="images/hosting_mcp_server.png" width="60%"/>
</div>

### チュートリアルの主な機能

* カスタムツールを使用した MCP サーバーの作成
* MCP サーバーのローカルテスト
* Amazon Bedrock AgentCore Runtime での MCP サーバーのホスティング
* 認証を使用したデプロイ済み MCP サーバーの呼び出し

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* 設定済みの AWS 認証情報
* Amazon Bedrock AgentCore SDK
* MCP（Model Context Protocol）ライブラリ
* 実行中の Docker

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## MCP（Model Context Protocol）の理解

MCP は AI モデルが外部データやツールに安全にアクセスすることを可能にするプロトコルです。主要な概念：

* **ツール**: AI がアクションを実行するために呼び出すことができる関数
* **Streamable HTTP**: AgentCore Runtime で使用される転送プロトコル
* **セッション分離**: 各クライアントは `Mcp-Session-Id` ヘッダーを通じて分離されたセッションを取得
* **ステートレス操作**: サーバーはスケーラビリティのためにステートレス操作をサポートする必要がある

AgentCore Runtime は、MCP サーバーがデフォルトパスとして `0.0.0.0:8000/mcp` でホストされることを期待します。

### プロジェクト構造

適切な構造でプロジェクトを設定しましょう：

```
mcp_server_project/
├── mcp_server.py              # メイン MCP サーバーコード
├── my_mcp_client.py          # ローカルテスト用クライアント
├── my_mcp_client_remote.py   # リモートテスト用クライアント
├── requirements.txt          # 依存関係
└── __init__.py              # Python パッケージマーカー
```

## MCP サーバーの作成

次に、3つのシンプルなツールを使用して MCP サーバーを作成しましょう。サーバーは AgentCore Runtime の互換性に必要な `stateless_http=True` を設定した FastMCP を使用します。

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

### このコードの動作

* **FastMCP**: ツールをホストできる MCP サーバーを作成
* **@mcp.tool()**: Python 関数を MCP ツールに変換するデコレーター
* **stateless_http=True**: AgentCore Runtime の互換性に必要
* **ツール**: さまざまなタイプの操作を示す3つのシンプルなツール

## ローカルテスト用クライアントの作成

AgentCore Runtime にデプロイする前に、MCP サーバーをローカルでテストするためのクライアントを作成しましょう：

In [ ]:
%%writefile my_mcp_client.py
import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

### ローカルでのテスト

MCP サーバーをローカルでテストするには：

1. **ターミナル 1**: MCP サーバーを起動
   ```bash
   python mcp_server.py
   ```
   
2. **ターミナル 2**: テストクライアントを実行
   ```bash
   python my_mcp_client.py
   ```

出力に3つのツールが表示されるはずです。

## 認証のための Amazon Cognito のセットアップ

AgentCore Runtime は認証を必要とします。デプロイされた MCP サーバーにアクセスするために JWT トークンを提供する Amazon Cognito を使用します。

In [ ]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

## AgentCore Runtime デプロイメントの設定

次に、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、および requirements ファイルを使用して AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて docker ファイルが生成されます。

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="mcp_server_agentcore"
)
print("Configuration completed ✓")

## MCP サーバーの AgentCore Runtime への起動

docker ファイルができたので、MCP サーバーを AgentCore Runtime に起動しましょう。これにより Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

## AgentCore Runtime ステータスの確認

AgentCore Runtime をデプロイしたので、デプロイメントステータスを確認し、準備完了まで待機しましょう：

In [ ]:
print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

## リモートアクセス用の設定の保存

デプロイされた MCP サーバーを呼び出す前に、簡単に取得できるように Agent ARN と Cognito 設定を AWS Systems Manager Parameter Store と AWS Secrets Manager に保存しましょう：

In [ ]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

## リモートテスト用クライアントの作成

次に、デプロイされた MCP サーバーをテストするためのクライアントを作成しましょう。このクライアントは AWS から必要な認証情報を取得し、デプロイされたサーバーに接続します：

In [ ]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## デプロイされた MCP サーバーのテスト

リモートクライアントを使用してデプロイされた MCP サーバーをテストしましょう：

In [ ]:
print("Testing deployed MCP server...")
print("=" * 50)
!python my_mcp_client_remote.py

## MCP ツールのリモート呼び出し

次に、ツールをリストするだけでなく、実際にそれらを呼び出して完全な MCP 機能を実証する拡張クライアントを作成しましょう：

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                
                try:
                    print("\n➕ Testing add_numbers(5, 3)...")
                    add_result = await session.call_tool(
                        name="add_numbers",
                        arguments={"a": 5, "b": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n✖️  Testing multiply_numbers(4, 7)...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers",
                        arguments={"a": 4, "b": 7}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n👋 Testing greet_user('Alice')...")
                    greet_result = await session.call_tool(
                        name="greet_user",
                        arguments={"name": "Alice"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                print("\n✅ MCP tool testing completed!")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## ツール呼び出しのテスト

実際にツールを呼び出して MCP ツールをテストしましょう：

In [ ]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

## 次のステップ

AgentCore Runtime に MCP サーバーを正常にデプロイできました。次のことができます：

1. **より多くのツールの追加**: 追加のツールで MCP サーバーを拡張
2. **カスタム認証**: カスタム JWT オーソライザーの実装
3. **統合**: 他の AgentCore サービスとの統合

## クリーンアップ（オプション）

このチュートリアルで作成されたリソースをクリーンアップしたい場合は、以下のセルを実行してください：

In [ ]:
import boto3

print("🗑️  Starting cleanup process...")

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    print("Deleting AgentCore Runtime...")
    runtime_delete_response = agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=launch_result.agent_id,
    )
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_repo_name = launch_result.ecr_uri.split('/')[1]
    ecr_client.delete_repository(
        repositoryName=ecr_repo_name,
        force=True
    )
    print("✓ ECR repository deleted")

    try:
        ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
        print("✓ Parameter Store parameter deleted")
    except ssm_client.exceptions.ParameterNotFound:
        print("ℹ️  Parameter Store parameter not found")
    try:
        secrets_client.delete_secret(
            SecretId='mcp_server/cognito/credentials',
            ForceDeleteWithoutRecovery=True
        )
        print("✓ Secrets Manager secret deleted")
    except secrets_client.exceptions.ResourceNotFoundException:
        print("ℹ️  Secrets Manager secret not found")

    print("\n✅ Cleanup completed successfully!")
    
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

# 🎉 おめでとうございます！

正常に完了しました：

✅ **カスタムツールを使用したMCPサーバーの作成**  
✅ **MCPクライアントを使用したローカルテスト**  
✅ **Amazon Cognitoによる認証のセットアップ**  
✅ **AgentCore Runtimeを使用したAWSへのデプロイ**  
✅ **適切な認証を使用したリモート呼び出し**  
✅ **MCPコンセプトとベストプラクティスの学習**  

MCP サーバーは現在 Amazon Bedrock AgentCore Runtime 上で実行されており、本番環境での使用が可能です！

## まとめ

このチュートリアルでは、以下を学習しました：
- FastMCP を使用した MCP サーバーの構築
- AgentCore 互換性のためのステートレス HTTP トランスポートの設定
- Amazon Cognito による JWT 認証のセットアップ
- AWS での MCP サーバーのデプロイと管理
- ローカルとリモートの両方でのテスト
- ツール呼び出しのための MCP クライアントの使用

デプロイされた MCP サーバーは、より大きな AI アプリケーションやワークフローに統合できるようになりました！